In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

import seaborn as sns
import nltk
from collections import Counter

In [2]:
data = pd.read_csv('conversation_data.csv', header = None)
data1 = pd.read_csv('convo_data_march.csv', header = None)
data

,0,1,2,3,4,5
0,CMPT_LEX_01,NaN,NaN,NaN,NaN,NaN
1,"[07:30, 13/04/2024",EcoEnthusiast123,Hi…. My vermicompost bin is infested with fru...,Question,NaN,NaN
2,"[09:32, 13/04/2024",GreenThumbGuru,"Fruit flies can be a nuisance, but there are ...",Answer,NaN,NaN
3,"[09:35, 13/04/2024",GreenThumbGuru,You can also try placing a small bowl of appl...,Answer,NaN,NaN
4,"[09:38, 13/04/2024",EcoEnthusiast123,I'll give that a try…. Anything else I can do?,Question,NaN,NaN
...,...,...,...,...,...,...
146,"[14:08, 12/04/2024",OrganicEnthusiast,That's fantastic! It's always rewarding to se...,Irrelevant,NaN,NaN
147,"[14:10, 12/04/2024",EcoFriendlyGardener,Absolutely! It's been such a rewarding journe...,Irrelevant,NaN,NaN
148,"[14:13, 12/04/2024",OrganicEnthusiast,I'm so glad to hear that. Keep up the great w...,Irrelevant,NaN,NaN
149,"[14:15, 12/04/2024",EcoFriendlyGardener,Will do! Thanks again for all your help and i...,Irrelevant,NaN,NaN


In [3]:
data = data.drop(columns=[4,5])
data = data[data[3]!="Irrelevant"]
data.reset_index(inplace=True)
data = data.drop(columns=['index'])
qa_pairs = []
convo_id = None
for index, row in data.iterrows():
    if isinstance(row[0], str) and "CMPT" in row[0]:
        convo_id = row[0]
    # Check if the current row is a question
    if row[3] == 'Question':
        question = row[2]
        answers = ""

        # Find all answers corresponding to the current question
        for i in range(index + 1, len(data)):
            if data.loc[i, 3] == 'Answer':
                answers += data.loc[i, 2] + " "  # Concatenate answers with a space
            else:
                break
        
        # Append the question with concatenated answers to qa_pairs
        qa_pairs.append({'Question': question, 'Answers': answers.strip(), 'Convo': convo_id})  # Remove trailing space

In [4]:
data1 = data1[data1[3]!="Irrelevant"]
data1.reset_index(inplace=True)
data1 = data1.drop(columns=['index'])
for index, row in data1.iterrows():
    if isinstance(row[0], str) and "CMPT" in row[0]:
        convo_id = row[0]
    # Check if the current row is a question
    if row[3] == 'Question':
        question = row[2]
        answers = ""

        # Find all answers corresponding to the current question
        for i in range(index + 1, len(data1)):
            if data1.loc[i, 3] == 'Answer':
                answers += data1.loc[i, 2] + " "  # Concatenate answers with a space
            else:
                break
        
        # Append the question with concatenated answers to qa_pairs
        qa_pairs.append({'Question': question, 'Answers': answers.strip(), 'Convo': convo_id+".1"})  # Remove trailing space

# Convert the list of dictionaries to a new dataframe
df = pd.DataFrame(qa_pairs)
df

,Question,Answers,Convo
0,Hi…. My vermicompost bin is infested with fru...,"Fruit flies can be a nuisance, but there are w...",CMPT_LEX_01
1,I'll give that a try…. Anything else I can do?,Make sure you're not overfeeding the bin with ...,CMPT_LEX_01
2,Why are there magoots in my compost bin??,Hey there! Maggots can sometimes appear in com...,CMPT_LEX_02
3,My vermicompost smells terrible! What should ...,"Hmm, that doesn't sound good. Foul odors usual...",CMPT_LEX_03
4,"Okay, I'll give that a try. Anything else?","Yes, if it's too wet, add more dry materials l...",CMPT_LEX_03
5,"Aiyah, my vermicompost bin got some problem lah.","Oh dear, what happened now?",CMPT_LEX_04
6,My kid accidentally poured too much water ins...,"Wah, must be quite soggy for the worms. Maybe ...",CMPT_LEX_04
7,"Eh bro, you ever hv problem with the browns t...","Hello yes, sometimes it's like juggling. Too m...",CMPT_LEX_05
8,"Wah, steady lah. Got any tricks or tips to sh...",Yes! I always keep some dried leaves or newspa...,CMPT_LEX_05
9,That's wonderful to hear! Keeping your worms ...,OK. I make sure to monitor the moisture levels...,CMPT_LEX_07


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Question  40 non-null     object
 1   Answers   40 non-null     object
 2   Convo     40 non-null     object
dtypes: object(3)
memory usage: 1.1+ KB


In [6]:
eda = df.copy()
eda['num_words_questions'] = eda['Question'].str.split().str.len()
eda['num_words_answers'] = eda['Answers'].str.split().str.len()
eda['num_vocab_question'] = eda['Question'].str.lower().str.split().apply(set).str.len()
eda['num_vocab_answer'] = eda['Answers'].str.lower().str.split().apply(set).str.len()
eda.describe()

,num_words_questions,num_words_answers,num_vocab_question,num_vocab_answer
count,40.000000,40.000000,40.000000,40.000000
mean,14.125000,31.025000,13.700000,27.575000
std,6.115082,25.353387,5.538675,18.912603
min,6.000000,5.000000,6.000000,5.000000
25%,9.750000,19.000000,9.750000,18.000000
50%,13.000000,25.000000,13.000000,24.000000
75%,18.000000,33.250000,17.000000,30.250000
max,31.000000,131.000000,30.000000,97.000000


### Cleaning

In [7]:
df.iloc[0,:]

Question     Hi…. My vermicompost bin is infested with fru...
Answers     Fruit flies can be a nuisance, but there are w...
Convo                                             CMPT_LEX_01
Name: 0, dtype: object

In [8]:
clean = df.copy()
clean['Question'] = clean['Question'].apply(lambda x: ''.join(char for char in x if char.isalnum() or char.isspace()))  # Remove punctuation
clean['Question'] = clean['Question'].apply(lambda x: x.strip())    # Remove leading and trailing whitespace
clean['Question'] = clean["Question"].apply(lambda x: x.replace("  ", " ")) # Remove double spaces
clean['Question'] = clean['Question'].apply(lambda x: x.lower())    # Convert text to lowercase
clean['Answers'] = clean['Answers'].apply(lambda x: ''.join(char for char in x if char.isalnum() or char.isspace()))
clean['Answers'] = clean['Answers'].apply(lambda x: x.strip()) 
clean['Answers'] = clean["Answers"].apply(lambda x: x.replace("  ", " ")) # Remove double spaces
clean['Answers'] = clean['Answers'].apply(lambda x: x.lower())    
clean

,Question,Answers,Convo
0,hi my vermicompost bin is infested with fruit ...,fruit flies can be a nuisance but there are wa...,CMPT_LEX_01
1,ill give that a try anything else i can do,make sure youre not overfeeding the bin with t...,CMPT_LEX_01
2,why are there magoots in my compost bin,hey there maggots can sometimes appear in comp...,CMPT_LEX_02
3,my vermicompost smells terrible what should i do,hmm that doesnt sound good foul odors usually ...,CMPT_LEX_03
4,okay ill give that a try anything else,yes if its too wet add more dry materials like...,CMPT_LEX_03
5,aiyah my vermicompost bin got some problem lah,oh dear what happened now,CMPT_LEX_04
6,my kid accidentally poured too much water insi...,wah must be quite soggy for the worms maybe tr...,CMPT_LEX_04
7,eh bro you ever hv problem with the browns to ...,hello yes sometimes its like juggling too much...,CMPT_LEX_05
8,wah steady lah got any tricks or tips to share,yes i always keep some dried leaves or newspap...,CMPT_LEX_05
9,thats wonderful to hear keeping your worms hea...,ok i make sure to monitor the moisture levels ...,CMPT_LEX_07


### Removing Stopwords (edit customized list as needed)

In [9]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
nltk.download('stopwords')

# Define custom stopwords
custom_stopwords = set(stopwords.words('english'))
custom = ['i', 'im', 'youre', 'oh', 'like']  # Add custom stopwords 
for c in custom:
    custom_stopwords.add(c)

# Function to remove stopwords from tokenized text
def remove_stopwords(text):
    text = [word for word in text.split() if word not in custom_stopwords]
    text = ' '.join(x for x in text)
    return text

xstopword = clean.copy()
xstopword['Question'] = clean['Question'].apply(remove_stopwords)
xstopword['Answers'] = clean['Answers'].apply(remove_stopwords)
xstopword


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fionn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Question,Answers,Convo
0,hi vermicompost bin infested fruit flies,fruit flies nuisance ways manage first make su...,CMPT_LEX_01
1,ill give try anything else,make sure overfeeding bin many wet food scraps...,CMPT_LEX_01
2,magoots compost bin,hey maggots sometimes appear compost bins cond...,CMPT_LEX_02
3,vermicompost smells terrible,hmm doesnt sound good foul odors usually indic...,CMPT_LEX_03
4,okay ill give try anything else,yes wet add dry materials shredded paper dried...,CMPT_LEX_03
5,aiyah vermicompost bin got problem lah,dear happened,CMPT_LEX_04
6,kid accidentally poured much water inside bin ...,wah must quite soggy worms maybe try drain exc...,CMPT_LEX_04
7,eh bro ever hv problem browns greens ratio com...,hello yes sometimes juggling much one thing smell,CMPT_LEX_05
8,wah steady lah got tricks tips share,yes always keep dried leaves newspaper standby...,CMPT_LEX_05
9,thats wonderful hear keeping worms healthy key...,ok make sure monitor moisture levels regularly...,CMPT_LEX_07


#### Bag of words after stopword removal (by label)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
# Questions
vectorizer = CountVectorizer(lowercase=True,
                             stop_words='english',
                             ngram_range=(1, 1),
                             min_df=0.05)
vectorizer.fit(xstopword['Question'])
X = vectorizer.transform(xstopword['Question'])
bow1 = vectorizer.get_feature_names_out()
bow1

array(['add', 'bedding', 'bin', 'browns', 'compost', 'dry', 'flies',
       'food', 'fruit', 'got', 'greens', 'healthy', 'hear', 'hey', 'hi',
       'ill', 'inside', 'ive', 'keeping', 'kid', 'lah', 'lately',
       'moisture', 'okay', 'problem', 'ratio', 'really', 'right',
       'scraps', 'secret', 'smells', 'sure', 'tank', 'thats', 'tips',
       'tricks', 'try', 'vermicompost', 'vermicomposting', 'wah', 'want',
       'waste', 'whats', 'wonderful', 'worm', 'worms', 'worried'],
      dtype=object)

In [11]:
# Answers
vectorizer = CountVectorizer(lowercase=True,
                             stop_words='english',
                             ngram_range=(1, 1),
                             min_df=0.05)
vectorizer.fit(xstopword['Answers'])
X = vectorizer.transform(xstopword['Answers'])
bow2 = vectorizer.get_feature_names_out()
bow2

array(['absorb', 'acidic', 'add', 'adding', 'adjust', 'air', 'airflow',
       'attracting', 'avoid', 'balance', 'balanced', 'bedding', 'bin',
       'bins', 'bury', 'cardboard', 'cause', 'compost', 'conditions',
       'cover', 'covered', 'crucial', 'dear', 'dont', 'dried', 'dry',
       'ensures', 'environment', 'excess', 'eye', 'flies', 'food',
       'foods', 'fruit', 'fruits', 'good', 'great', 'happened', 'healthy',
       'help', 'helps', 'hi', 'holes', 'indicate', 'keeping', 'layer',
       'leaves', 'levels', 'lightly', 'maintaining', 'make', 'materials',
       'maybe', 'mix', 'moist', 'moisture', 'new', 'newspaper', 'odors',
       'oily', 'overfeeding', 'oxygen', 'paper', 'prevent', 'proper',
       'properly', 'quite', 'ratio', 'regularly', 'remove', 'right',
       'scraps', 'sensitive', 'shredded', 'skin', 'smart', 'smells',
       'stick', 'sure', 'temperature', 'thing', 'tips', 'try', 'types',
       'unpleasant', 'usually', 'waste', 'water', 'wet', 'work', 'worm',
    

In [12]:
# Combined
set1 = set(bow1)
set2 = set(bow2)

result_array = np.array(list(set1.union(set2)))
result_array

array(['thats', 'work', 'compost', 'oxygen', 'thing', 'whats',
       'newspaper', 'eye', 'overfeeding', 'waste', 'tank', 'browns',
       'hey', 'good', 'tips', 'lightly', 'maintaining', 'secret',
       'worried', 'levels', 'holes', 'problem', 'leaves', 'cause',
       'acidic', 'dont', 'adjust', 'crucial', 'food', 'shredded', 'wet',
       'indicate', 'ill', 'regularly', 'keeping', 'ratio', 'new', 'got',
       'make', 'greens', 'absorb', 'remove', 'maybe', 'helps', 'prevent',
       'hear', 'air', 'cover', 'smart', 'sure', 'fruits', 'excess',
       'layer', 'vermicomposting', 'help', 'fruit', 'sensitive',
       'usually', 'airflow', 'bins', 'great', 'right', 'water', 'okay',
       'bury', 'paper', 'tricks', 'foods', 'lah', 'moisture',
       'vermicompost', 'hi', 'inside', 'avoid', 'cardboard', 'flies',
       'environment', 'materials', 'want', 'try', 'adding', 'moist',
       'smells', 'bin', 'ive', 'proper', 'dry', 'types', 'really',
       'unpleasant', 'quite', 'wah', 'happ

### Stemming

#### Snowball

In [13]:
snowball = xstopword.copy()
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
def stemm_words(text):
    text = [stemmer.stem(word) for word in text.split()]
    text = ' '.join(x for x in text)
    return text
snowball['text'] = snowball['text'].apply(stemm_words)
snowball['text'][27]

KeyError: 'text'

In [ ]:
# Questions
vectorizer = CountVectorizer(lowercase=True,
                             stop_words='english',
                             ngram_range=(1, 1),
                             min_df=0.05)
vectorizer.fit(snowball[snowball['label'] == "Question"]["text"])
X = vectorizer.transform(snowball[snowball['label'] == "Question"]["text"])
vectorizer.get_feature_names_out()

array(['add', 'bin', 'brown', 'compost', 'dri', 'fli', 'food', 'fruit',
       'got', 'green', 'healthi', 'ill', 'kid', 'late', 'moistur', 'okay',
       'problem', 'ratio', 'realli', 'right', 'smell', 'tip', 'tri',
       'vermicompost', 'want', 'worm'], dtype=object)

In [ ]:
# Answer
vectorizer = CountVectorizer(lowercase=True,
                             stop_words='english',
                             ngram_range=(1, 1),
                             min_df=0.05)
vectorizer.fit(snowball[snowball['label'] == "Answer"]["text"])
X = vectorizer.transform(snowball[snowball['label'] == "Answer"]["text"])
vectorizer.get_feature_names_out()

array(['absorb', 'acid', 'ad', 'add', 'adjust', 'aerat', 'airflow',
       'attract', 'avoid', 'balanc', 'bed', 'bin', 'buri', 'cardboard',
       'compost', 'condit', 'cover', 'dont', 'dri', 'ensur', 'environ',
       'excess', 'eye', 'fli', 'food', 'fruit', 'good', 'happen', 'help',
       'layer', 'leav', 'level', 'maintain', 'make', 'materi', 'moistur',
       'new', 'newspap', 'odor', 'prevent', 'proper', 'remov', 'scrap',
       'shred', 'smell', 'stick', 'sure', 'temperatur', 'tip', 'tri',
       'type', 'unpleas', 'wast', 'wet', 'work', 'worm', 'yes'],
      dtype=object)

#### Porter

In [ ]:
porter = xstopword.copy()
from nltk.stem.snowball import PorterStemmer
stemmer = PorterStemmer()
def stemm_words(text):
    text = [stemmer.stem(word) for word in text.split()]
    text = ' '.join(x for x in text)
    return text
porter['text'] = porter['text'].apply(stemm_words)
porter['text'][27]

'havent realli monitor condit close bz work ah'

In [ ]:
# Questions
vectorizer = CountVectorizer(lowercase=True,
                             stop_words='english',
                             ngram_range=(1, 1),
                             min_df=0.05)
vectorizer.fit(porter[porter['label'] == "Question"]["text"])
X = vectorizer.transform(porter[porter['label'] == "Question"]["text"])
vectorizer.get_feature_names_out()

array(['add', 'bin', 'brown', 'compost', 'dri', 'fli', 'food', 'fruit',
       'got', 'green', 'healthi', 'ill', 'kid', 'late', 'moistur', 'okay',
       'problem', 'ratio', 'realli', 'right', 'smell', 'tip', 'tri',
       'vermicompost', 'want', 'worm'], dtype=object)

In [ ]:
# Answer
vectorizer = CountVectorizer(lowercase=True,
                             stop_words='english',
                             ngram_range=(1, 1),
                             min_df=0.05)
vectorizer.fit(porter[porter['label'] == "Answer"]["text"])
X = vectorizer.transform(porter[porter['label'] == "Answer"]["text"])
vectorizer.get_feature_names_out()

array(['absorb', 'acid', 'ad', 'add', 'adjust', 'aerat', 'airflow',
       'attract', 'avoid', 'balanc', 'bed', 'bin', 'buri', 'cardboard',
       'compost', 'condit', 'cover', 'dont', 'dri', 'ensur', 'environ',
       'excess', 'eye', 'fli', 'food', 'fruit', 'good', 'happen', 'help',
       'layer', 'leav', 'level', 'maintain', 'make', 'materi', 'moistur',
       'new', 'newspap', 'odor', 'prevent', 'proper', 'remov', 'scrap',
       'shred', 'smell', 'stick', 'sure', 'temperatur', 'tip', 'tri',
       'type', 'unpleas', 'wast', 'wet', 'work', 'worm', 'ye'],
      dtype=object)

#### Lancaster Stemmer

In [ ]:
lc = xstopword.copy()
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
def stemm_words(text):
    text = [stemmer.stem(word) for word in text.split()]
    text = ' '.join(x for x in text)
    return text
lc['text'] = lc['text'].apply(stemm_words)
lc['text'][27]

'hav real monit condit clos bz work ah'

In [ ]:
# Questions
vectorizer = CountVectorizer(lowercase=True,
                             stop_words='english',
                             ngram_range=(1, 1),
                             min_df=0.05)
vectorizer.fit(lc[lc['label'] == "Question"]["text"])
X = vectorizer.transform(lc[lc['label'] == "Question"]["text"])
vectorizer.get_feature_names_out()

array(['ad', 'bin', 'brown', 'compost', 'dry', 'fli', 'food', 'fruit',
       'giv', 'got', 'green', 'healthy', 'il', 'kid', 'lat', 'moist',
       'okay', 'problem', 'ratio', 'real', 'right', 'smel', 'tip', 'try',
       'vermicompost', 'want', 'worm'], dtype=object)

In [ ]:
# Answer
vectorizer = CountVectorizer(lowercase=True,
                             stop_words='english',
                             ngram_range=(1, 1),
                             min_df=0.05)
vectorizer.fit(lc[lc['label'] == "Answer"]["text"])
X = vectorizer.transform(lc[lc['label'] == "Answer"]["text"])
vectorizer.get_feature_names_out()

array(['absorb', 'acid', 'ad', 'adjust', 'aer', 'airflow', 'attract',
       'avoid', 'bal', 'bed', 'bin', 'bury', 'cardboard', 'compost',
       'condit', 'cov', 'dont', 'dri', 'dry', 'ens', 'environ', 'excess',
       'ey', 'fli', 'food', 'fruit', 'good', 'gre', 'hap', 'help', 'lay',
       'leav', 'level', 'maintain', 'mak', 'mat', 'moist', 'new',
       'newspap', 'od', 'prev', 'prop', 'remov', 'scraps', 'shredded',
       'smel', 'stick', 'sur', 'temp', 'tip', 'try', 'typ', 'unpleas',
       'wast', 'wel', 'wet', 'work', 'worm', 'ye'], dtype=object)

### Lemmatization

In [14]:
lemm = xstopword.copy()
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemm_words(text):
    text = [lemmatizer.lemmatize(word) for word in text.split()]
    text = ' '.join(x for x in text)
    return text
lemm['Question'] = lemm['Question'].apply(lemm_words)
lemm['Answers'] = lemm['Answers'].apply(lemm_words)
lemm

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fionn\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\fionn\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,Question,Answers,Convo
0,hi vermicompost bin infested fruit fly,fruit fly nuisance way manage first make sure ...,CMPT_LEX_01
1,ill give try anything else,make sure overfeeding bin many wet food scrap ...,CMPT_LEX_01
2,magoots compost bin,hey maggot sometimes appear compost bin condit...,CMPT_LEX_02
3,vermicompost smell terrible,hmm doesnt sound good foul odor usually indica...,CMPT_LEX_03
4,okay ill give try anything else,yes wet add dry material shredded paper dried ...,CMPT_LEX_03
5,aiyah vermicompost bin got problem lah,dear happened,CMPT_LEX_04
6,kid accidentally poured much water inside bin ...,wah must quite soggy worm maybe try drain exce...,CMPT_LEX_04
7,eh bro ever hv problem brown green ratio compo...,hello yes sometimes juggling much one thing smell,CMPT_LEX_05
8,wah steady lah got trick tip share,yes always keep dried leaf newspaper standby r...,CMPT_LEX_05
9,thats wonderful hear keeping worm healthy key ...,ok make sure monitor moisture level regularly ...,CMPT_LEX_07


In [15]:
# Answers
vectorizer = CountVectorizer(lowercase=True,
                             stop_words='english',
                             ngram_range=(1, 1),
                             min_df=0.05)
vectorizer.fit(lemm['Question'])
X = vectorizer.transform(lemm['Question'])
bow1 = vectorizer.get_feature_names_out()
bow1

array(['add', 'bedding', 'bin', 'brown', 'compost', 'dry', 'fly', 'food',
       'fruit', 'got', 'green', 'healthy', 'hear', 'hey', 'hi', 'ill',
       'inside', 'ive', 'keeping', 'kid', 'lah', 'lately', 'moisture',
       'okay', 'problem', 'ratio', 'really', 'right', 'scrap', 'secret',
       'smell', 'sure', 'tank', 'thats', 'tip', 'trick', 'try',
       'vermicompost', 'vermicomposting', 'wah', 'want', 'waste', 'whats',
       'wonderful', 'worm', 'worried'], dtype=object)

In [16]:
# Answers
vectorizer = CountVectorizer(lowercase=True,
                             stop_words='english',
                             ngram_range=(1, 1),
                             min_df=0.05)
combined_text = lemm['Question'] + ' ' + lemm['Answers']

# Fit and transform combined text
vectorized_combined = vectorizer.fit_transform(combined_text).toarray()

# Convert to DataFrame
bow = pd.DataFrame(vectorized_combined, columns=vectorizer.get_feature_names_out())
bow

,absorb,acidic,add,adding,adjust,ah,air,airflow,attention,attracting,...,waste,water,way,wet,whats,wonderful,work,worm,worried,yes
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,0,1,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,1,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,1,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,0,1,0,1,0,0,0,0,1,0,...,0,0,0,1,1,1,2,7,0,0


### Exporting file

In [17]:
lemm

,Question,Answers,Convo
0,hi vermicompost bin infested fruit fly,fruit fly nuisance way manage first make sure ...,CMPT_LEX_01
1,ill give try anything else,make sure overfeeding bin many wet food scrap ...,CMPT_LEX_01
2,magoots compost bin,hey maggot sometimes appear compost bin condit...,CMPT_LEX_02
3,vermicompost smell terrible,hmm doesnt sound good foul odor usually indica...,CMPT_LEX_03
4,okay ill give try anything else,yes wet add dry material shredded paper dried ...,CMPT_LEX_03
5,aiyah vermicompost bin got problem lah,dear happened,CMPT_LEX_04
6,kid accidentally poured much water inside bin ...,wah must quite soggy worm maybe try drain exce...,CMPT_LEX_04
7,eh bro ever hv problem brown green ratio compo...,hello yes sometimes juggling much one thing smell,CMPT_LEX_05
8,wah steady lah got trick tip share,yes always keep dried leaf newspaper standby r...,CMPT_LEX_05
9,thats wonderful hear keeping worm healthy key ...,ok make sure monitor moisture level regularly ...,CMPT_LEX_07


In [18]:
processed_data = []
grouped = lemm.groupby('Convo')
for convo, group in grouped:
    conversation = ""
    for index, row in group.iterrows():
        # Concatenate question and answer with prefixes
        pair = "### Human: " + row["Question"] + "### Assistant: " + row["Answers"]
        conversation += pair + " "
    processed_data.append(conversation)

# Create a new DataFrame from the processed data
processed_df = pd.DataFrame({"text": processed_data})

# Print the processed DataFrame
processed_df

# Export DataFrame to CSV
processed_df.to_csv("final.csv", index=False)

In [19]:
processed_df

,text
0,### Human: hi vermicompost bin infested fruit ...
1,### Human: tank smell v badd### Assistant: don...
2,### Human: magoots compost bin### Assistant: h...
3,### Human: worm bin smell really bad### Assist...
4,### Human: vermicompost smell terrible### Assi...
5,### Human: hey new took ur class last wk sure ...
6,### Human: aiyah vermicompost bin got problem ...
7,### Human: want start vermicomposting session ...
8,### Human: eh bro ever hv problem brown green ...
9,### Human: think worm bin wet dry there moistu...


### Tokenization

In [ ]:
from llama.generation import Llama, Dialog
from llama.model import ModelArgs, Transformer
from llama.tokenizer import Tokenizer

# Initialize the Llama tokenizer
tokenizer = Tokenizer(model_path="C:\\Users\\fionn\\Course\\ASG2\\DSCP_grpA\\compost\\tokenizer.model")

def tokenize_text(text):
    # Tokenize the preprocessed text using the Llama tokenizer"
    tokens = tokenizer.encode(text, bos=True, eos=True)
    return tokens

tokenized = lemm.copy()
tokenized['Question'] = tokenized['Question'].apply(tokenize_text)
tokenized['Answers'] = tokenized['Answers'].apply(tokenize_text)
tokenized

,Question,Answers
0,"([1, 7251, 12108, 293, 290, 2490, 9016, 3041, ...","([1, 15774, 11340, 302, 4664, 749, 982, 10933,..."
1,"([1, 4486, 2367, 1018, 3099, 1683, 2], [1, 1, ...","([1, 1207, 1854, 975, 18798, 292, 9016, 1784, ..."
2,"([1, 2320, 29877, 1862, 752, 520, 9016, 2], [1...","([1, 540, 29891, 2320, 7085, 6041, 2615, 752, ..."
3,"([1, 12108, 293, 290, 2490, 1560, 514, 16403, ...","([1, 298, 4317, 19403, 6047, 1781, 285, 5059, ..."
4,"([1, 20759, 4486, 2367, 1018, 3099, 1683, 2], ...","([1, 4874, 7990, 788, 15589, 5518, 528, 1127, ..."
5,"([1, 7468, 29891, 801, 12108, 293, 290, 2490, ...","([1, 9425, 9559, 2], [1, 1, 1, 1])"
6,"([1, 26397, 11423, 635, 7446, 1127, 1568, 4094...","([1, 281, 801, 1818, 3755, 17846, 1927, 281, 5..."
7,"([1, 321, 29882, 2545, 3926, 298, 29894, 1108,...","([1, 22172, 4874, 6041, 8740, 29887, 1847, 156..."
8,"([1, 281, 801, 27357, 425, 29882, 2355, 8938, ...","([1, 4874, 2337, 3013, 270, 1255, 20447, 19656..."
9,"([1, 20952, 20695, 8293, 12515, 281, 555, 9045...","([1, 3431, 1207, 1854, 11819, 2730, 391, 545, ..."


### Feature Extraction (Try word embedding or contextual embedding)